## Topic Modeling of Social Media Data regarding the 2019-nCoV Pandemic.
#### Data source: Twitter
#### Last data update: 01/28/2020 - 16:17

In [1]:
import json
import gensim
import nltk
import pandas as pd
import pickle
import pyLDAvis.gensim
import random
import spacy
from gensim import corpora
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
spacy.load('en_core_web_sm')
from spacy.lang.en import English
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/zero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/zero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
with open('datasets/data.json') as f:
    data = json.load(f)
data_list = []
for item in data:
    data_list.append(item['text'])
data_series = pd.Series(data_list)
data_series.head(5)

0    @lookner I taught a class in college on the co...
1                   this corona virus is scary as hell
2                     new corona virus is pretty scary
3    If Corona has a Virus?  I'm a dead man..  pic....
4    Corona Beer claims a conspiracy is behind the ...
dtype: object

### Data preprocessing

#### Text cleaning

In [3]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            pass
        elif token.orth_.startswith('@'):
            pass
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

en_stop = set(nltk.corpus.stopwords.words('english'))

In [4]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [5]:
text_data = []

for line in data_list:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        text_data.append(tokens)    

#### LDA

In [6]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('artifacts/corpus.pkl', 'wb'))
dictionary.save('artifacts/dictionary.gensim')

In [7]:
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=50,  alpha=80)
ldamodel.save('artifacts/model.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.061*"virus" + 0.057*"corona" + 0.013*"drink" + 0.011*"infect" + 0.010*"thought" + 0.010*"come" + 0.009*"wearing" + 0.008*"everyone" + 0.008*"america" + 0.008*"cough"')
(1, '0.112*"corona" + 0.104*"virus" + 0.012*"disease" + 0.010*"chinese" + 0.010*"china" + 0.009*"horrible" + 0.009*"south" + 0.009*"sorry" + 0.008*"alarm" + 0.008*"please"')
(2, '0.092*"virus" + 0.092*"corona" + 0.026*"china" + 0.014*"coronavirus" + 0.013*"spreading" + 0.012*"crown" + 0.011*"hospital" + 0.009*"chinese" + 0.009*"outbreak" + 0.008*"quarantine"')
(3, '0.095*"corona" + 0.083*"virus" + 0.014*"wuhan" + 0.013*"hate" + 0.012*"pandemic" + 0.010*"take" + 0.009*"coronavirus" + 0.009*"outbreak" + 0.009*"update" + 0.009*"video"')
(4, '0.113*"virus" + 0.089*"corona" + 0.014*"people" + 0.013*"worst" + 0.013*"first" + 0.012*"hate" + 0.009*"call" + 0.009*"think" + 0.009*"transport" + 0.008*"china"')


In [8]:
print(ldamodel.print_topics(num_topics=5, num_words=3))

[(0, '0.061*"virus" + 0.057*"corona" + 0.013*"drink"'), (1, '0.112*"corona" + 0.104*"virus" + 0.012*"disease"'), (2, '0.092*"virus" + 0.092*"corona" + 0.026*"china"'), (3, '0.095*"corona" + 0.083*"virus" + 0.014*"wuhan"'), (4, '0.113*"virus" + 0.089*"corona" + 0.014*"people"')]


In [9]:
from gensim.models.coherencemodel import CoherenceModel

print('\nPerplexity: ', ldamodel.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(model=ldamodel, texts=text_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.513059028921256

Coherence Score:  0.6518706496073154


#### LDA visualization

In [ ]:
dictionary = gensim.corpora.Dictionary.load('artifacts/dictionary.gensim')
corpus = pickle.load(open('artifacts/corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('artifacts/model.gensim')
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)